In [1]:
import re
import locale
import pandas as pd
import requests
import os

from webLib import textFromURLorFile
from bs4 import BeautifulSoup

# pd.set_option('display.max_rows', 100)

# Configura o locale para o formato de número brasileiro
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

'pt_BR.utf8'

In [5]:
url = 'https://sisgvarmazenamento.blob.core.windows.net/prd/PublicacaoPortal/Arquivos/202306.htm'
texto = textFromURLorFile(url, "dados-202306.txt")

# https://regex101.com/

for resultado in re.finditer(r"Vereador\(a\): (( |\.|\w+)*)\n(.+?)(?=Vereador)", texto, re.DOTALL):
    nome_vereador = resultado.group(1)
    dados = texto[resultado.start():resultado.end()]
    if "VEREADOR AFASTADO" in dados:
        continue

    total = re.search(r"TOTAL DO MÊS\s+([\d.,]+)", dados)
    valor = locale.atof(total.group(1))
    print("\n", valor, nome_vereador)

    for itens in re.finditer(r"(?P<item>.*)\s*(?P<subitem>(([\d.\/-]+)\s(.*)\s([\d.,]+)\s*)+)TOTAL DO ITEM\s+(?P<total>[\d.,]+)", dados):
        item = itens.group("item")
        valor = locale.atof(itens.group("total"))
        print("\t", valor, item)


 18122.71 ADILSON AMADEU
	 13000.0 COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA
	 1002.44 INTERMEDIADO - CORREIOS
	 2757.18 INTERMEDIADO - LOCAÇÃO DE VEÍCULOS
	 257.59 TELEFONE FIXO
	 1105.5 TELEFONE MOVEL

 23509.54 ALESSANDRO GUEDES
	 454.59 COMBUSTIVEL
	 4000.0 COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA
	 14133.28 CONTRATAÇAO DE PESSOA JURIDICA
	 2757.18 INTERMEDIADO - LOCAÇÃO DE VEÍCULOS
	 1710.0 LOCAÇÃO DE MOVEIS E EQUIPAMENTOS
	 192.63 TELEFONE FIXO
	 261.86 TELEFONE MOVEL

 18960.05 ANDRÉ SANTOS
	 1009.14 COMBUSTIVEL
	 2995.0 COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA
	 10699.0 CONTRATAÇAO DE PESSOA JURIDICA
	 3641.91 INTERMEDIADO - LOCAÇÃO VEICULOS HIBRIDOS
	 615.0 LOCAÇÃO DE MOVEIS E EQUIPAMENTOS

 17351.03 ARSELINO TATTO
	 926.23 COMBUSTIVEL
	 4700.0 COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA
	 3500.0 ELABORAÇÃO/MANUTENÇAO DE SITE/HOSPEDAGEM
	 16.37 INTERMEDIADO - CORREIOS
	 3641.91 INTERMEDIADO - LOCAÇÃO VEICULOS HIBRIDOS
	 4200

In [15]:
ano = {}
for n in range(1, 7, 1):
    url = f'https://sisgvarmazenamento.blob.core.windows.net/prd/PublicacaoPortal/Arquivos/2023{n:02d}.htm'
    texto = textFromURLorFile(url, f"dados-2023{n:02d}.txt")

    for resultado in re.finditer(r"Vereador\(a\): (( |\.|\w+)*)\n(.+?)(?=Vereador)", texto, re.DOTALL):
        nome_vereador = resultado.group(1)
        dados = texto[resultado.start():resultado.end()]
        if "VEREADOR AFASTADO" in dados:
            continue

        total = re.search(r"TOTAL DO MÊS\s+([\d.,]+)", dados)
        valor = locale.atof(total.group(1))
        if nome_vereador not in ano:
            ano[nome_vereador] = {"total": 0}
            
        ano[nome_vereador]["total"] += valor

        for itens in re.finditer(r"(?P<item>.*)\s*(?P<subitem>(([\d.\/-]+)\s(.*)\s([\d.,]+)\s*)+)TOTAL DO ITEM\s+(?P<total>[\d.,]+)", dados):
            item = itens.group("item")
            valor = locale.atof(itens.group("total"))
            if item not in ano[nome_vereador]:
                ano[nome_vereador][item] = 0
            
            ano[nome_vereador][item] += valor

ano

{'ADILSON AMADEU': {'total': 89835.63,
  'INTERMEDIADO - LOCAÇÃO DE VEÍCULOS': 16543.079999999998,
  'TELEFONE FIXO': 1464.1499999999999,
  'TELEFONE MOVEL': 6290.32,
  'COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA': 21000.0,
  'INTERMEDIADO - CORREIOS': 44538.08},
 'ALESSANDRO GUEDES': {'total': 161796.29,
  'COMBUSTIVEL': 3644.1300000000006,
  'COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA': 49084.75,
  'CONTRATAÇAO DE PESSOA JURIDICA': 78068.48000000001,
  'INTERMEDIADO - LOCAÇÃO DE VEÍCULOS': 16543.079999999998,
  'LOCAÇÃO DE MOVEIS E EQUIPAMENTOS': 10260.0,
  'TELEFONE FIXO': 1135.44,
  'TELEFONE MOVEL': 1595.1600000000003,
  'ELABORAÇÃO/MANUTENÇAO DE SITE/HOSPEDAGEM': 144.35,
  'MATERIAL DE ESCRITORIO E OUTROS MATERIAIS DE CONSUMO': 1320.9},
 'ANDRÉ SANTOS': {'total': 130577.28,
  'COMBUSTIVEL': 7685.76,
  'COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA': 12975.0,
  'CONTRATAÇAO DE PESSOA JURIDICA': 79126.0,
  'INTERMEDIADO - LOCAÇÃO VEICULOS HIBRIDOS':

In [17]:
df = pd.DataFrame.from_dict(ano, orient='index')
df.reset_index(inplace=True)
df.rename(columns={'index': 'vereador'}, inplace=True)
df.fillna(0, inplace=True)
df = df.sort_values(by='total', ascending=False)
df = df.reset_index(drop=True)
df.to_csv('sp-totais.csv', index=False)
df

,vereador,total,INTERMEDIADO - LOCAÇÃO DE VEÍCULOS,TELEFONE FIXO,TELEFONE MOVEL,COMPOSIÇÃO/ARTE/DIAGRAMAÇÃO/PRODUÇÃO/IMPRESSAO GRAFICA,INTERMEDIADO - CORREIOS,COMBUSTIVEL,CONTRATAÇAO DE PESSOA JURIDICA,LOCAÇÃO DE MOVEIS E EQUIPAMENTOS,...,PROVEDOR DE INTERNET,LIMPEZA DE VEICULOS,REPROGRAFIA (XEROX/ENCADERNAÇÃO),ASSINATURA DE JORNAIS E REVISTAS,CORREIOS,INTERMEDIADO - REPROGRAFIA (XEROX/ENCADERNAÇÃO),LIVROS,APERFEIÇOAMENTO PROFISSIONAL,LOCAÇÃO DE VEICULOS HIBRIDOS,LOCAÇÃO DE VEÍCULOS
0,EDIR SALES,192712.50,0.00,1379.09,0.0,25701.71,39073.06,4966.49,72714.53,6900.00,...,0.00,0.0,2588.5,0.0,8844.30,0.0,0.0,0.0,0.0,0.0
1,LUANA ALVES,191212.90,16543.08,0.00,0.0,24700.00,0.00,7165.59,91767.03,26790.00,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,RODOLFO DESPACHANTE,191021.15,14153.52,0.00,0.0,47745.00,0.00,6668.13,89200.00,10650.00,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
3,JOÃO JORGE,189836.88,0.00,0.00,0.0,20585.00,0.00,10830.11,89149.91,25800.00,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,ISAC FELIX,187560.81,16543.08,0.00,0.0,35520.00,0.00,12668.49,41920.00,32478.81,...,0.00,0.0,0.0,0.0,29811.41,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,ERIKA HILTON,17609.53,0.00,0.00,0.0,0.00,13490.30,477.32,0.00,0.00,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
60,FELIPE BECARI,14800.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
61,DELEGADO PALUMBO,13167.42,2757.18,0.00,0.0,0.00,0.00,0.00,10000.00,0.00,...,63.24,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
62,JOSÉ POLICE NETO,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
